<a href="https://colab.research.google.com/github/mhanauer/pg_report/blob/main/pg_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/mhanauer/pg_report.git

Cloning into 'pg_report'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [3]:
!pip install xgboost
!pip install lightgbm
!pip install scikit-optimize
import pandas as pd
import sklearn
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import numpy as np
from joblib import dump, load
from sklearn.model_selection import RepeatedKFold
import plotly.express as px
%cd pg_report/

     |████████████████████████████████| 100 kB 7.1 MB/s 
/content/pg_report


Step: Load in data
(Please present this answer visually)
How does our conversion rate change through the funnel overall and by marketing channels?

Create overall conversation rate which is the percentage of sign_and_exam to started navigator.

Then analyze this conversation rate by marketing change.




In [4]:
funnel_path = pd.read_csv("funnel_path.csv")
users = pd.read_csv("users.csv")
funnel_path_users = funnel_path.merge(users, on='user_id', how='left')
funnel_path_users


,user_id,funnel_steps,age,device,income,num_conditions,premium,marketing_channels
0,1001,collect_contact_info,56,desktop,157490,5,2490.123774,instagram
1,1001,started_navigator,56,desktop,157490,5,2490.123774,instagram
2,1002,application_submit,34,desktop,138171,3,1906.228442,paid_search_nb
3,1002,collect_contact_info,34,desktop,138171,3,1906.228442,paid_search_nb
4,1002,phone_connect,34,desktop,138171,3,1906.228442,paid_search_nb
...,...,...,...,...,...,...,...,...
27624,10999,collect_contact_info,20,mobile_ios,163641,4,1786.076634,podcast
27625,10999,started_navigator,20,mobile_ios,163641,4,1786.076634,podcast
27626,10999,viewed_quotes,20,mobile_ios,163641,4,1786.076634,podcast
27627,11000,collect_contact_info,21,mobile_ios,169217,3,1756.614104,podcast


Calculate conversation rate overall and by channel

In [5]:
funnel_path_users

,user_id,funnel_steps,age,device,income,num_conditions,premium,marketing_channels
0,1001,collect_contact_info,56,desktop,157490,5,2490.123774,instagram
1,1001,started_navigator,56,desktop,157490,5,2490.123774,instagram
2,1002,application_submit,34,desktop,138171,3,1906.228442,paid_search_nb
3,1002,collect_contact_info,34,desktop,138171,3,1906.228442,paid_search_nb
4,1002,phone_connect,34,desktop,138171,3,1906.228442,paid_search_nb
...,...,...,...,...,...,...,...,...
27624,10999,collect_contact_info,20,mobile_ios,163641,4,1786.076634,podcast
27625,10999,started_navigator,20,mobile_ios,163641,4,1786.076634,podcast
27626,10999,viewed_quotes,20,mobile_ios,163641,4,1786.076634,podcast
27627,11000,collect_contact_info,21,mobile_ios,169217,3,1756.614104,podcast


Step: 

In [50]:
number_of_customers = len(pd.unique(funnel_path_users['user_id']))
number_of_customers

10000

In [54]:
def count_marketing_channel_fun(data, var1, var2): 
  test = data[["funnel_steps", "marketing_channels"]].value_counts().reset_index(name='count')
  test[["percentage"]] = test[["count"]] / number_of_customers
  test[["percentage"]] = test[["percentage"]].round(2) 
  test = test[test.funnel_steps == "sign_and_exam"]
  return test

count_marketing_channel = count_marketing_channel_fun(funnel_path_users, "funnel_steps", "marketing_channels")
count_marketing_channel

,funnel_steps,marketing_channels,count,percentage
19,sign_and_exam,paid_search_nb,303,0.03
22,sign_and_exam,podcast,222,0.02
27,sign_and_exam,instagram,106,0.01
28,sign_and_exam,facebook,97,0.01
29,sign_and_exam,tv,56,0.01


Conversation rate overall   

In [72]:
def count_percentage(data, var):
  test = data[[var]].value_counts().reset_index(name='count')
  test[["percentage"]] = test[["count"]] / number_of_customers
  test[["percentage"]] = test[["percentage"]].round(2)
  test = test[test[var] == 'sign_and_exam']
  test.insert(1, 'marketing_channels', 'overall')
  return test
overall_conversion_rate = count_percentage(data = funnel_path_users, var = "funnel_steps")
overall_conversion_rate

,funnel_steps,marketing_channels,count,percentage
5,sign_and_exam,overall,784,0.08


Step: Combine the overall with the marketing conversation rates.

In [75]:
overall_marketing_conversation_rates = pd.concat([overall_conversion_rate, count_marketing_channel], axis = 0)
overall_marketing_conversation_rates

,funnel_steps,marketing_channels,count,percentage
5,sign_and_exam,overall,784,0.08
19,sign_and_exam,paid_search_nb,303,0.03
22,sign_and_exam,podcast,222,0.02
27,sign_and_exam,instagram,106,0.01
28,sign_and_exam,facebook,97,0.01
29,sign_and_exam,tv,56,0.01


Step: Plot bar graph with minimal background and only marketing channel axes.

In [98]:
overall_marketing_conversation_rates['marketing_channels'] = overall_marketing_conversation_rates['marketing_channels'].astype('category')
overall_marketing_conversation_rates[["marketing_channels"]] = overall_marketing_conversation_rates[["marketing_channels"]].replace(["overall", "paid_search_nb", "podcast", "instagram", "facebook", "tv"], ["Overall", "Paid search nb", "Podcast", "Instagram", "Facebook", "TV"])
overall_marketing_conversation_rates


,funnel_steps,marketing_channels,count,percentage
5,sign_and_exam,Overall,784,0.08
19,sign_and_exam,Paid search nb,303,0.03
22,sign_and_exam,Podcast,222,0.02
27,sign_and_exam,Instagram,106,0.01
28,sign_and_exam,Facebook,97,0.01
29,sign_and_exam,TV,56,0.01


In [103]:
fig = px.bar(overall_marketing_conversation_rates, x='percentage', y='marketing_channels', text_auto=True, orientation='h', title="Overall and by marketing channel conversation rates")
fig.update_layout(yaxis={'categoryorder':'total ascending'}) # add only this line

fig.update_xaxes(title_text = "")
fig.update_yaxes(title_text = "")

fig.update_xaxes(showticklabels=False)

# Makes the background transparent
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

fig.show()